<a href="https://colab.research.google.com/github/varsha03-xjk/NLP/blob/main/nlp2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers gradio accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [2]:
from transformers import pipeline

# Load the FLAN Alpaca Large model
paraphraser = pipeline("text2text-generation", model="declare-lab/flan-alpaca-large")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [3]:
!pip install transformers gradio sentencepiece

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Load model and tokenizer
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define style prompts
def generate_prompt(text, style):
    style_prompts = {
        "Formal": "Please rewrite the following text in a formal and professional tone:\n\n",
        "Friendly": "Please rewrite the following text in a casual and friendly tone:\n\n",
        "Poetic": "Rewrite the text in a poetic and metaphorical way, like a short verse:\n\n",
        "Gen Z": "Rewrite this text using Gen Z slang, internet expressions, abbreviations, and emojis:\n\n"
    }

    # Default if style not found
    base_prompt = style_prompts.get(style, "Rewrite the text:\n\n")
    return base_prompt + text

# Function to rewrite text in selected style
def rewrite_text(text, style):
    if not text.strip():
        return "Please enter some text."

    # Build the prompt based on the selected style
    if style == "Gen Z":
        prompt = f"Rewrite the following text in a funny Gen Z tone with slang, emojis, and internet expressions:\n\n{text}"
    elif style == "Poetic":
        prompt = f"Rewrite the following text in a poetic and artistic style:\n\n{text}"
    elif style == "Formal":
        prompt = f"Rewrite the following text in a formal, professional tone:\n\n{text}"
    elif style == "Friendly":
        prompt = f"Rewrite the following text in a friendly and conversational style:\n\n{text}"
    else:
        prompt = f"Rewrite the following text:\n\n{text}"

    # Call the model (paraphraser)
    response = paraphraser(prompt, max_length=100)[0]['generated_text']

    # Optional: clean output (remove repeated prompt from response if needed)
    return response.replace(prompt, "").strip()


# Gradio UI
import gradio as gr

# Define the interface
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown(
        """
        <h1 style="text-align: center;">📝 Rewrite My Text</h1>
        <p style="text-align: center;">Transform your text into <b>fun, formal, poetic, or Gen Z</b> styles using AI! 🚀</p>
        """,
        elem_id="header",
    )

    with gr.Row():
        with gr.Column(scale=1):
            input_text = gr.Textbox(
                label="Input Text",
                placeholder="Enter your sentence here...",
                lines=4
            )
            style = gr.Dropdown(
                label="Choose Style",
                choices=["Gen Z", "Formal", "Poetic", "Friendly"],
                value="Gen Z"
            )
            submit_button = gr.Button("✨ Submit", variant="primary")
            clear_button = gr.Button("🧹 Clear")

        with gr.Column(scale=1):
            output_text = gr.Textbox(
                label="Rewritten Text",
                placeholder="Your rewritten sentence will appear here...",
                lines=4
            )

    # Button functionality
    submit_button.click(fn=rewrite_text, inputs=[input_text, style], outputs=output_text)
    clear_button.click(fn=lambda: ("", ""), inputs=[], outputs=[input_text, output_text])

# Launch the app
demo.launch(share=True)



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4e31caec19a48f7ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
